In [7]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
from tqdm import tqdm
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D, StarDist3D

import os
import cv2
import pandas as pd 
import pathlib

from joblib import Parallel, delayed
import multiprocessing

from skimage.transform import resize
from skimage import io   

import json

In [58]:
model_folder = r'D:\Mirjam\3D nuclear seg+quant pipeline tutorials and files\files\models\3Dmodel_organoid(Leah)'
directory = r'D:\Fabien\test\CRC13_Dapi-a18-H3K9me23-LaminB'
segmentation = '3D'
seg_channel = 0 
n_tiles = [1,3,3] #2D:[xtiles,ytiles], 3D:[ztiles,xtiles,ytiles]
prob = 0.004
nms = 0.004

In [59]:
np.random.seed(6)
lbl_cmap = random_label_cmap()


tresh_set = {"prob": prob, "nms": nms}

with open(model_folder + '\\thresholds.json', 'w') as outfile:
    json.dump(tresh_set, outfile)  

model_folder = pathlib.PurePath(model_folder)

if segmentation == '2D':
    model = StarDist2D(None, name=model_folder.name, basedir=str(model_folder.parents[0]))
if segmentation == '3D':
    model = StarDist3D(None, name=model_folder.name, basedir=str(model_folder.parents[0]))
    
folders = glob(directory + "/*/")

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.004, nms_thresh=0.004.


In [44]:
def compute_folder(folder, segmentation, n_tiles):
    
    img_folder = folder + subfolder
    img_files =  glob(folder + '\\' + subfolder + '\\*.tif')
    
    if segmentation == '2D':
        output_folder = folder + 'rois\\'
        if not os.path.isdir(folder + 'rois'):
            os.mkdir(output_folder)
    
    if segmentation == '3D':
        output_folder = folder + 'masks\\'
        if not os.path.isdir(folder + 'masks'):
            os.mkdir(output_folder)

    for img_file in img_files:
        
        print(img_file)
        
        image = imread(img_file) 
        if segmentation == '2D':
            if len(image.shape) == 3:
                image = image[seg_channel,::] 
            
            axis_norm = (0,1)   # normalize channels independently
            image = normalize(image, 1,99.8, axis=axis_norm)
            labels, polygons = model.predict_instances(image)
            export_imagej_rois(output_folder + pathlib.PurePath(img_file).name.replace('.tif', '_roi.zip'),
                               polygons['coord'], n_tiles=n_tiles)  
                           
        if segmentation == '3D':
            if len(image.shape) == 4:
                image = image[seg_channel,::]
                
            axis_norm = (0,1,2)
            labels, details = model.predict_instances(image, n_tiles=n_tiles)
            save_tiff_imagej_compatible(output_folder + pathlib.PurePath(img_file).name.replace('.tif', '_mask.tif'), 
                                        labels.astype(np.int16), axes='ZYX')           
            

In [45]:
for folder in folders:
    compute_folder(folder, segmentation, n_tiles)

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

D:\Fabien\test\CRC13_Dapi-a18-H3K9me23-LaminB\AKPS\AKPS-Dapi-a18-H3K9me23-LaminB-1-2.tif


  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

D:\Fabien\test\CRC13_Dapi-a18-H3K9me23-LaminB\AKPS\AKPS-Dapi-a18-H3K9me23-LaminB-1.tif


  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

...

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

D:\Fabien\test\CRC13_Dapi-a18-H3K9me23-LaminB\WT\WT-Dapi-a18-H3K9me23-LaminB-3-2.tif


  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

D:\Fabien\test\CRC13_Dapi-a18-H3K9me23-LaminB\WT\WT-Dapi-a18-H3K9me23-LaminB-3.tif


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  5.07it/s]
